[ref](https://classroom.udacity.com/courses/cs373)

In [1]:
import numpy as np

In [25]:
p = [0.2, 0.2, 0.2, 0.2, 0.2]
world = ['green', 'red', 'red', 'green', 'green']
Z = 'red'
pHit = 0.6
pMiss = 0.2

Bayes Rule
---
$$ P(A|B) = \frac{P(A)P(B|A)}{P(B)} \hspace{1cm}(1) $$

$$ P(X_2|red) = \frac{P(X_2)P(red|X_2)}{P(red)} \hspace{1cm}(2) $$

$$ P(red) = \sum_{i=0}^4P(X_i)P(red|X_i) \hspace{1cm}(3) $$

In [13]:
def sense(p, Z):
    prob = np.array(p)
    measure = np.array([pHit if i == Z else pMiss for i in world])
    
    combine_prob = prob*measure
    
    norm_prob = combine_prob/sum(combine_prob)
    return norm_prob

In [11]:
sense(p, Z)

array([0.11111111, 0.33333333, 0.33333333, 0.11111111, 0.11111111])

In [26]:
measurements = ['red', 'green']

for i in measurements:
    p = sense(p, i)
    print("sensing [{}] -> prior distribution {}".format(i, p))

sensing [red] -> prior distribution [0.11111111 0.33333333 0.33333333 0.11111111 0.11111111]
sensing [green] -> prior distribution [0.2 0.2 0.2 0.2 0.2]


In [34]:
def move(p, U):
    """
    p: probability distribution 
    U: number of steps moving
    """
    n = len(p)
    U = U % n
    
    return p[-U:] + p[:(n-U)]

In [37]:
p = [0.11111111, 0.33333333, 0.33333333, 0.11111111, 0.11111111]

move(p, -1)

[0.33333333, 0.33333333, 0.11111111, 0.11111111, 0.11111111]

### Non-accurate Move

$$ P(X_j) = \sum_{i}P(X_i)P(X_j|X_i) \hspace{1cm}(4) $$

In [39]:
pExact = 0.8
pOvershoot = 0.1
pUndershoot = 0.1

In [44]:
def move(p, U):
    """
    movement inaccurate
    p: probability distribution 
    U: number of steps moving
    """
    p = np.array(p)
    move_prob = np.array([pOvershoot, pExact, pUndershoot])
    n = len(p)
    U = U % n
    
    q = []
    for i in range(n):
        steps = [i-U-1, i-U, i-U+1]
        q_prob = np.dot(p[steps], move_prob)
        q.append(q_prob)
    return q

In [46]:
p = [0, 1, 0, 0, 0]
move(p, 1)

[0.0, 0.1, 0.8, 0.1, 0.0]

In [48]:
# when move infinite number of times, the distribution would go normal
p = [0, 1, 0, 0, 0]
n = 100
U = 2

for _ in range(n):
    p = move(p, U)

print(p)

[0.20000004294005816, 0.20000013895694374, 0.20000004294005816, 0.19999988758147377, 0.19999988758147375]


Sense & Move Cycle
---
In effect, this is `Bayes Probability` + `Total Probability`
<img src="sense-move.png" width="600">

In [52]:
p = [0.2, 0.2, 0.2, 0.2, 0.2]
world = ['green', 'red', 'red', 'green', 'green']
measurements = ['red', 'green']
motions = [1, 1]
pHit = 0.6
pMiss = 0.2
pExact = 0.8
pOvershoot = 0.1
pUndershoot = 0.1

In [49]:
def move(p, U):
    """
    movement inaccurate
    p: probability distribution 
    U: number of steps moving
    """
    p = np.array(p)
    move_prob = np.array([pOvershoot, pExact, pUndershoot])
    n = len(p)
    U = U % n
    
    q = []
    for i in range(n):
        steps = [i-U-1, i-U, i-U+1]
        q_prob = np.dot(p[steps], move_prob)
        q.append(q_prob)
    return q

In [50]:
def sense(p, Z):
    prob = np.array(p)
    measure = np.array([pHit if i == Z else pMiss for i in world])
    
    combine_prob = prob*measure
    
    norm_prob = combine_prob/sum(combine_prob)
    return norm_prob

In [53]:
# first sense red
p = sense(p, "red")
# move right
p = move(p, 1)
# second sense green
p = sense(p, "green")
# move right again
p = move(p, 1)

print(p)

[0.21157894736842103, 0.1515789473684211, 0.08105263157894739, 0.16842105263157897, 0.3873684210526316]


In [54]:
measurements = ["red", "red"]
motions = [1, 1]
p = [0.2, 0.2, 0.2, 0.2, 0.2]

for i in range(len(measurements)):
    # sense -> move
    p = sense(p, measurements[i])
    p = move(p, motions[i])
print(p)

[0.07882352941176471, 0.07529411764705884, 0.22470588235294123, 0.4329411764705882, 0.18823529411764706]
